In [52]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon, chi2
import statistics as st
%matplotlib inline

LOC = 0
SCALE = 5
N = 100
L = 10

ɑ = 0.1

rng = np.random.default_rng(81381)
dist = LOC + rng.exponential(scale=SCALE, size=N)

# Критерий Χ^2 Пирсона

In [54]:
def chi2_pirson():
	hist, bin_edges = np.histogram(dist, bins=L)
	bin_centers = bin_edges[:-1] + np.diff(bin_edges) / 2

	mx = 1 / N * sum([hist[i] * bin_centers[i] for i in range(0, L)])
	Dx = 1 / (N - 1) * sum(
		[hist[i] * pow(bin_centers[i] - mx, 2) for i in range(0, L)]
	)
	σ = pow(Dx, 0.5)

	z = abs(bin_centers - mx) / σ
	_np = N * (bin_edges[1] - bin_edges[0]) / σ * expon.pdf(z)

	_chi2 = sum(pow(hist - _np, 2) / _np)
	print(_chi2)

	return (mx, Dx, σ)


print( chi2_pirson() )
print(
	st.mean(dist),
	st.variance(dist),
	st.stdev(dist)
)

62.35990444889791
(5.731496902589677, 32.23936101544494, 5.677971558175062)
5.326778188251635 33.416027553602184 5.780659785318816
